### Types:
 - Content based 
 - Colloborative based
     - User Based
     - Item Based
           
- Recommendation Engines can be Supervised/Unsupervised
- Hybrid models as well are there

In [4]:
import numpy as np
import pandas as pd

In [6]:
movies = pd.read_csv('D:/Datasets/movies/movies.csv')
ratings = pd.read_csv('D:/Datasets/movies/ratings.csv')
#ratings .head()
#movies.head()

ratings['userId'] = 'User-'+ ratings['userId'].astype(str)
ratings['movieId'] = 'Movie-' + ratings['movieId'].astype(str)
ratings.head()

,userId,movieId,rating
0,User-1,Movie-31,2.5
1,User-1,Movie-1029,3.0
2,User-1,Movie-1061,3.0
3,User-1,Movie-1129,2.0
4,User-1,Movie-1172,4.0


### Create user-movie table

In [7]:
print('No.of.Unique Movies %d' % ratings['movieId'].nunique())
print('No.of.Unique users %d' % ratings['userId'].nunique())

No.of.Unique Movies 9066
No.of.Unique users 671


In [10]:
user_item_matrix = ratings.pivot_table(index = 'userId',columns='movieId',values='rating')
print(user_item_matrix.shape)
user_item_matrix.head()

(671, 9066)


movieId,Movie-1,Movie-10,Movie-100,Movie-100017,Movie-100032,Movie-100034,Movie-100083,Movie-100106,Movie-100159,Movie-100163,...,Movie-99795,Movie-998,Movie-99811,Movie-99813,Movie-99839,Movie-99846,Movie-999,Movie-99912,Movie-99917,Movie-99992
userId,,,,,,,,,,,,,,,,,,,,,
User-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
User-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
User-100,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
User-101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
User-102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
# 1 . Average Number of Ratings per user

#user_item_matrix.apply(lambda v : len(v) - pd.isnull(v).sum(),axis = 1)
user_item_matrix.count(axis = 1).head() # counting Row wise so axis = 1
print(user_item_matrix.count(axis = 1).mean()) # mean of the movies which were rated

print('-----')

# 2 . User with Most number of ratings 
print(user_item_matrix.count(axis = 1).sort_values(ascending = False).head(1))

print('-----')

# 3. Average number of ratings per movie

print(user_item_matrix.count().mean())

print('-----')

# 4.Movie with most number of ratings (most discussed movies)

print(user_item_matrix.count().sort_values(ascending = False).head(1))

149.03725782414307
-----
userId
User-547    2391
dtype: int64
-----
11.030664019413193
-----
movieId
Movie-356    341
dtype: int64


In [28]:
# Cosine Similarity
from sklearn.metrics.pairwise import cosine_similarity

- Use Cosine Similarity as a measure for distance between 2 neighbors when large dimensions of data is there.
 instead of Euclidean Distance
- cos 0 , if a and b are close to each other the angle between them is 0 then cos 0 = 1
- cos 90 if a and b are orthogonal to each other the angle between them is 90 then cos 90 is 0 
- cos 180 if a and b are 180 degree then cos 180= -1

- Need to do missing value imputation inorder to apply cosine similarity
- Higher the cosine similarity between 2 users, higher the similarity between users.
- If 2 movies are to be recommended , best among them is chosen by the highest rating among 2
- Angle between users is 0 , the similarity will be higher

-Objective:
    - Take user 100 , pair with all other unique users whcih users have the highest similarity that is the neighbor pair.

- Need to impute the data as Cosine similarity wont work if ther are missing values.
- so normalize the data , take the row sum subtract with each and every cell , then impute with 0 .

In [32]:
# subtracting the mean from every cell so to normalize the data
#ui_matrix  = user_item_matrix.copy()
# No need to copy , changes are not geting affected
ui_matrix = user_item_matrix.apply(lambda v : v - v.mean(),axis = 1).fillna(0)
ui_matrix.head()

movieId,Movie-1,Movie-10,Movie-100,Movie-100017,Movie-100032,Movie-100034,Movie-100083,Movie-100106,Movie-100159,Movie-100163,...,Movie-99795,Movie-998,Movie-99811,Movie-99813,Movie-99839,Movie-99846,Movie-999,Movie-99912,Movie-99917,Movie-99992
userId,,,,,,,,,,,,,,,,,,,,,
User-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
User-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
User-100,0.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
User-101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
User-102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
# cosine Similaity
sim_matrix = pd.DataFrame(cosine_similarity(ui_matrix),index = ui_matrix.index,columns=ui_matrix.index)
sim_matrix.head(1)
# when matrix has + ve values , cosine similarity wont go below 0 .

userId,User-1,User-10,User-100,User-101,User-102,User-103,User-104,User-105,User-106,User-107,...,User-90,User-91,User-92,User-93,User-94,User-95,User-96,User-97,User-98,User-99
userId,,,,,,,,,,,,,,,,,,,,,
User-1,1.0,0.0,0.0,0.0,-0.021761,0.013193,0.0,-0.017057,0.0,0.0,...,0.00556,0.0,0.0,0.0,0.002487,-0.016758,0.002888,0.0,0.033916,-0.003196


In [41]:
# For User-100
neigh_user100 = sim_matrix['User-100'].drop('User-100').sort_values(ascending = False).head(7).index
# Most similar user to user 100 is user 636 then 459 then 25 and so on.
neigh_user100

Index(['User-636', 'User-459', 'User-25', 'User-112', 'User-18', 'User-70',
       'User-357'],
      dtype='object', name='userId')

In [45]:
# take the user 100 alone
# user_item_matrix.loc['User-100']

# Pulling out the cells that are not rated by user 100

#DataFrame
preds = pd.DataFrame(columns=['mid','predicted_rating'])
for mid,rating in user_item_matrix.loc['User-100'].items():  # gets both index as well as value
    if pd.isnull(rating):
        #print(mid)
        mid_ratings = user_item_matrix.loc[neigh_user100,mid]
        avg_mid_rating = mid_ratings.mean()
        preds = preds.append({'mid':mid,'predicted_rating':avg_mid_rating},ignore_index = True)
        #print(mid,mid_ratings.values,avg_mid_rating)

preds['movieId'] = preds['mid'].str.replace('Movie-','').astype(int)

preds = pd.merge(preds,movies,left_on = 'movieId',right_on = 'movieId')
preds.sort_values('predicted_rating',ascending = False).head(3)['title']

6186                        Beauty and the Beast (1991)
6316    Homeward Bound II: Lost in San Francisco (1996)
6363                                   Jane Eyre (1996)
Name: title, dtype: object

In [49]:
# Creating a User Defined function

#DataFrame
def ubcf(userid):
    preds = pd.DataFrame(columns=['mid','predicted_rating'])
    neigh_user = sim_matrix[userid].drop(userid).sort_values(ascending = False).head(7).index
    for mid,rating in user_item_matrix.loc[userid].items():  # gets both index as well as value
        if pd.isnull(rating):
            #print(mid)
            mid_ratings = user_item_matrix.loc[neigh_user,mid]
            avg_mid_rating = mid_ratings.mean()
            preds = preds.append({'mid':mid,'predicted_rating':avg_mid_rating},ignore_index = True)
            #print(mid,mid_ratings.values,avg_mid_rating)

    preds['movieId'] = preds['mid'].str.replace('Movie-','').astype(int)

    preds = pd.merge(preds,movies,left_on = 'movieId',right_on = 'movieId')
    movies_recom = preds.sort_values('predicted_rating',ascending = False).head(3)['title']
    return movies_recom

In [50]:
ubcf('User-1')

7285    Lord of the Rings: The Return of the King, The...
734                                Terminator, The (1984)
814                             This Is Spinal Tap (1984)
Name: title, dtype: object

### Scraping  and Recomm Engines

In [51]:
! pip install selenium

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [1]:
from selenium import webdriver
browser = webdriver.Chrome('chromedriver.exe')

In [2]:
zomato = "https://www.zomato.com/bangalore/red-rhino-whitefield"
browser.get(zomato)

In [3]:
while True:
    more_sel = "div.load-more"
    try:
        load_more_button = browser.find_element_by_css_selector(more_sel)
        load_more_button.click()
        #selector = browser.find_element_by_css_selector('div.ui.segment.clearfix.brtop')
    except:
        break

In [8]:
df_rest = pd.DataFrame(columns=['UserId','UserName','Rating'])

try:
    selector = 'div.ui.segment.clearfix.brtop'

except:
    selector = 'div.ui.segment.clearfix.br0'
    
parents = browser.find_elements_by_css_selector(selector)
for parent in parents:
    name_selec = 'a[data-entity_id]'
    user = parent.find_element_by_css_selector(name_selec)
    user_id = user.get_attribute('data-entity_id')
    user_name = user.get_attribute('innerHTML')
    rating = parent.find_element_by_css_selector('div[aria-label]')
    rating = rating.get_attribute('aria-label')
    #print(user,user_id,user_name,rating)
    zom_rest = {
        'UserId': user_id,
        'UserName': user_name,
        'Rating':rating
    }
    
    df_rest = df_rest.append(zom_rest, ignore_index=True)
df_rest

,UserId,UserName,Rating
0,48713273,\n Ignatius Francis (Engineeroffood...,Rated 5.0
1,62399836,\n {((=TUMPA=))}\n,Rated 5.0
2,16250035,\n Sarthak Kumar\n,Rated 4.0
3,16368124,\n Mamatha Jayaram\n,Rated 5.0
4,35260907,\n Hemanth R\n,Rated 5.0
5,3110745,\n Shreya Murthy\n,Rated 4.0
6,16206054,\n Anubhav Routray\n,Rated 4.0
7,136738,\n Konali Gupta\n,Rated 5.0
8,1249295,\n Rachna Shivale\n,Rated 4.0
9,425231,\n Neha Primith\n,Rated 4.0
